In [1]:
import os
import sys

home=os.path.expanduser("~")
os.environ["SPARK_HOME"]=os.path.join(home, 'spark-2.0.0-bin-hadoop2.7')
os.environ["PYLIB"]=os.path.join(os.environ["SPARK_HOME"],'python','lib')
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'py4j-0.10.1-src.zip'))
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'pyspark.zip'))

In [2]:
import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder.master("local").appName("myApp").config(conf=myConf).getOrCreate()

# 실습 - 연설문 

In [35]:
import os
from pyspark.sql.types import StructType, StructField, StringType
police=spark.read\
    .option("header","true")\
    .option("delimiter"," ")\
    .option("inferSchema","true")\
    .schema(
        StructType([
            StructField("sent",StringType()),
            ])
    )\
    .text(os.path.join("data", "20191021_policeAddress.txt")) 

from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="sent", outputCol="words")
tokDf = tokenizer.transform(police)

from pyspark.ml.feature import StopWordsRemover
stop = StopWordsRemover(inputCol="words", outputCol="nostops")
stop.setStopWords([u'그',u'더', u'참', u'등',u'될',u'돌',u'수',u'큰', u'나', u'너', u'우리'])

stopDf=stop.transform(tokDf)

from pyspark.ml.feature import HashingTF, IDF

hashTF = HashingTF(inputCol="nostops", outputCol="hash", numFeatures=50)
hashDf = hashTF.transform(stopDf)
idf = IDF(inputCol="hash", outputCol="idf")
idfModel = idf.fit(hashDf)
idfDf = idfModel.transform(hashDf)

for e in idfDf.select("nostops","hash").collect():
    dd = e.asDict()
    for elem in dd.get('nostops'):
        print elem,
    print dd.get('hash')

존경하는 국민 여러분, 경찰관 여러분, 일흔네 ‘경찰의 날’입니다. (50,[9,15,17,23,36,39,46],[2.0,1.0,1.0,1.0,1.0,1.0,1.0])
(50,[],[])
국민의 안전을 위해 밤낮없이 애쓰시는 전국의 15만 경찰관 여러분께 먼저 감사를 드립니다. 전몰·순직 경찰관들의 고귀한 희생에 경의를 표합니다. 유가족 여러분께 위로의 마음을 전합니다. (50,[0,1,3,4,7,12,14,18,20,21,24,25,26,30,31,33,34,35,43,46],[1.0,1.0,3.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])
(50,[],[])
오늘 홍조근정훈장을 받으신 중앙경찰학교장 이은정 치안감님, 근정포장을 받으신 광주남부경찰서 김동현 경감님을 비롯한 수상자 여러분께 각별한 축하와 감사를 드립니다. 또한 경찰 영웅으로 추서되신 차일혁, 최중락님께 국민의 사랑을 전해드립니다. (50,[1,2,3,7,8,11,18,20,28,31,33,39,42,43,44,47,48,49],[1.0,1.0,3.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,3.0,2.0,1.0,1.0,2.0,1.0])
(50,[],[])
사랑하는 경찰관 여러분, (50,[9,24,46],[1.0,1.0,1.0])
(50,[],[])
여러분의 헌신적 노력으로 우리의 치안은 좋아졌습니다. 지난해 범죄 발생은 2015년에 비해 15.1% 줄었습니다. 같은 기간 교통사고 사망자는 18.2% 감소했습니다. (50,[0,5,13,15,16,23,27,28,29,30,31,36,39,46,48],[1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0])
(50,[],[])
치안의 개선은 국민의 체감으로 나타나고 있습니다. 올해 상반기 국민의 체감안전도는 74.5점으로 역대 최고를 기록했습니다. 범죄안전도는 처음으로 80점을 넘었습